In [75]:
# Do not edit this cell

# course: 3654
# a: Project 1
# d: VT

# S&P 500 Companies Pre/Post Covid

### Team: Rich Homies (Group 29)

**Name:**  Thejus Poruthikode Unnivelan
**PID:**  thejuspu

**Name:**  Andrew Visocan
**PID:**  avisocan
    
We have neither given nor received unauthorized assistance on this assignment. See the course sylabus for details on the Honor Code policy. In particular, sharing lines of solution code is prohibited.

Since we are a smaller group, we have coordinated to work on the project at the same time. The work has been done with collaboration on GitHub.

In [77]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from alpha_vantage.timeseries import TimeSeries   # pip install alpha_vantage in Anaconda Prompt
from sklearn import preprocessing
from datetime import datetime
import time

## Objective

To organize the companies on the S&P 500 into groups that flourished or struggled in the times of Covid-19, based on OHLCV data (Open, High, Low, Close, Volume). companyFile is a file containing details on the 509 companies on the S&P500, including ticker symbol, company name, their sector, and their industry (individual industries make up sectors).

In [59]:
companyFile = "S&P500_Companies.csv"
AVkey="ZH9AWDIE9WH36H30"
companies = pd.read_csv(companyFile)
companies.columns

Index(['Ticker', 'Company', 'Sector', 'Industry'], dtype='object')

Using one datapoint (AAL) we were able to create trimData which filters the dataframe from AlphaVantage to dates between the date 2 quarters before the crash date and 2 quarters after the crash date. A quarter is 3 months, so calculating the start and end date was pretty easy. We also found the pre-crash high and the post-crash low by looking at the S&P500 graph for reference. 

In [70]:
crash = datetime(2020, 2, 20)
start = datetime(2019, 8, 20)
end = datetime(2020, 8, 20)
preCrashHigh = (2020, 2, 20)
postCrashLow = (2020, 3, 23)

In [71]:
def trimData(data):
    data = data.loc[start:end, :]
    data = data.filter(['1. open', '2. high', '3. low', '4. close', '6. volume'], axis=1)
    data.columns = ["Open", "High", "Low", "Close", "Volume"]
    return data

We are limited to 5 calls a minute and 500 calls a day. So saving the data for later reading will be immensely useful. The second cell down reads the first 500 data points, and the third cell down reads the last 9 data points. Haven't collected all the data-files because it will take multiple days.

In [74]:
import os
os.makedirs('DATA')

In [78]:
count = 0
for symbol in companies.loc[0:499, 'Ticker']:
    ts = TimeSeries(key=AVkey, output_format='pandas')
    data, meta_data = ts.get_daily_adjusted(symbol, outputsize='full')
    trimData(data).to_csv('DATA/' + symbol + '.csv')
    count += 1
    if count == 5:
        count = 0
        time.sleep(60)

KeyboardInterrupt: 

In [82]:
count = 0
for symbol in companies.loc[500:, 'Ticker']:
    ts = TimeSeries(key=AVkey, output_format='pandas')
    data, meta_data = ts.get_daily_adjusted(symbol, outputsize='full')
    trimData(data).to_csv('DATA/' + symbol + '.csv')
    count += 1
    if count == 5:
        count = 0
        time.sleep(60)